In [1]:
import os
import lamini

lamini.api_key = 'c1edfdec132835d207702ea85fd5f2bbba2071528ccd2c48acefa689fccfbc2d'

In [2]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import Trainer
from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

In [3]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [4]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    #print(model)
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message.content

In [6]:
import regex as re
import tiktoken

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling

**Train dataset and model**

In [8]:
train_dataset_pdf = '../../bussines_helper_0.3/ISO9001.pdf'

In [9]:
%store -r titles_with_text

In [10]:
model_name = 'EleutherAI/pythia-410m'

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [12]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [13]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [14]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=200):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

**Generate questions**

In [15]:
question = 'How should top leadership demonstrate leadership and commitment?'

In [16]:
print(inference(question, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




The answer is simple: it’s not about the leader. It’s about the leader’s leadership.

Leadership is about the leader’s commitment to the mission.

Leadership is about the leader’s commitment to the people.

Leadership is about the leader’s commitment to the organization.

Leadership is about the leader’s commitment to the organization’s mission.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the


In [17]:
titles_with_text['Risk-based thinking'][1]

'The concept of risk-based thinking has been implicit in previous editions of this International Standard,  e.g. through requirements for planning, review and improvement. This International Standard  specifies requirements for the organization to understand its context (see  4.1 ) and determine risks as  a basis for planning (see  6.1 ). This represents the application of risk-based thinking to planning and  implementing quality management system processes (see  4.4 ) and will assist in determining the extent  of documented information. One of the key purposes of a quality management system is to act as a preventive tool. Consequently,  this International Standard does not have a separate clause or subclause on preventive action. The  concept of preventive action is expressed through the use of risk-based thinking in formulating quality  management system requirements. The risk-based thinking applied in this International Standard has enabled some reduction in  prescriptive requiremen

In [18]:
prompt = f"""
    Here is the text:
    {titles_with_text['Risk-based thinking'][1]}
    Ask 3 most relevant questions about this text only
"""

In [19]:
all_messages = [{'role' : 'user', 'content' : prompt}]

In [20]:
all_messages

[{'role': 'user',
  'content': '\n    Here is the text:\n    The concept of risk-based thinking has been implicit in previous editions of this International Standard,  e.g. through requirements for planning, review and improvement. This International Standard  specifies requirements for the organization to understand its context (see  4.1 ) and determine risks as  a basis for planning (see  6.1 ). This represents the application of risk-based thinking to planning and  implementing quality management system processes (see  4.4 ) and will assist in determining the extent  of documented information. One of the key purposes of a quality management system is to act as a preventive tool. Consequently,  this International Standard does not have a separate clause or subclause on preventive action. The  concept of preventive action is expressed through the use of risk-based thinking in formulating quality  management system requirements. The risk-based thinking applied in this International Sta

In [21]:
raw_questions = get_completion_from_messages(all_messages)
print(raw_questions)

1. What is the concept of risk-based thinking and how is it applied in this International Standard?
2. Is there a requirement for formal methods for risk management or a documented risk management process in this International Standard?
3. How does this International Standard provide greater flexibility compared to ISO 9001:2008 in terms of processes, documented information, and organizational responsibilities?


In [22]:
pattern = '\n?\d\. '
splits = re.split(pattern, raw_questions)
prepared_questions = splits[1:]
prepared_questions

['What is the concept of risk-based thinking and how is it applied in this International Standard?',
 'Is there a requirement for formal methods for risk management or a documented risk management process in this International Standard?',
 'How does this International Standard provide greater flexibility compared to ISO 9001:2008 in terms of processes, documented information, and organizational responsibilities?']

In [23]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [24]:
sum = 0
for title in list(titles_with_text.keys()):
    for i in range(len(titles_with_text[title])):
        print(title)
        text_token_length =  num_tokens_from_string(titles_with_text[title][i], "cl100k_base")
        
        if text_token_length == 0:
            print()
            continue
            
        n = (text_token_length // 100)**2 + 2
        print('n: ', n, '\n')
        sum += n
print(sum)

Foreword
n:  18 

Introduction

Scope
n:  3 

Normative references
n:  2 

Terms and definitions
n:  2 

Context of the organization

Understanding the organization and its context
n:  3 

Understanding the needs and expectations of interested parties
n:  2 

Understanding the needs and expectations of interested parties
n:  3 

Determining the scope of the quality management system
n:  6 

Quality management system and its processes
n:  6 

Leadership

Leadership and commitment

General
n:  18 

General
n:  11 

General
n:  6 

General
n:  2 

General
n:  3 

General
n:  2 

General
n:  3 

General
n:  2 

General
n:  2 

General
n:  2 

Customer focus
n:  2 

Policy

Establishing the quality policy
n:  2 

Communicating the quality policy
n:  2 

Organizational roles, responsibilities and authorities
n:  3 

Planning

Actions to address risks and opportunities
n:  6 

Quality objectives and planning to achieve them
n:  3 

Planning of changes
n:  2 

Support

Resources

People
n:  2 

In [25]:
def get_number_of_questions_for_title(text):
    text_token_length =  num_tokens_from_string(text, "cl100k_base")
    #print(title, ': ', text_token_length)

    if text_token_length == 0:
        return -1

    n = (text_token_length // 100)**2 + 2
    return n

In [26]:
titles_with_text['Determining the requirements for products and services']

['When determining the requirements for the products and services to be offered to customers, the  organization shall ensure that: a) the requirements for the products and services are defined, including: 1) any applicable statutory and regulatory requirements; 2) those considered necessary by the organization; b) the organization can meet the claims for the products and services it offers. ']

In [27]:
all_questions = []

for title in list(titles_with_text.keys()):
    for i in range(len(titles_with_text[title])):
        print(title)
        
        n = get_number_of_questions_for_title(titles_with_text[title][i])
        if n == -1:
            print()
            continue
        print('n: ', n)
        
        question_prompt = f"""
            Here is the text:
            {titles_with_text[title][i]}
            Ask {n} most relevant questions about this text only
        """
        
        raw_questions = get_completion(question_prompt)
        pattern = '\n?\d+[\.)] '
        splits = re.split(pattern, raw_questions)
        prepared_questions = splits[1:]

        print('Num of questions: ', len(prepared_questions))
       
        if(n != len(prepared_questions)):
            print(raw_questions)
        print()

        all_questions += prepared_questions

Foreword
n:  18
Num of questions:  18

Introduction

Scope
n:  3
Num of questions:  3

Normative references
n:  2
Num of questions:  2

Terms and definitions
n:  2
Num of questions:  2

Context of the organization

Understanding the organization and its context
n:  3
Num of questions:  3

Understanding the needs and expectations of interested parties
n:  2
Num of questions:  2

Understanding the needs and expectations of interested parties
n:  3
Num of questions:  3

Determining the scope of the quality management system
n:  6
Num of questions:  6

Quality management system and its processes
n:  6
Num of questions:  6

Leadership

Leadership and commitment

General
n:  18
Num of questions:  18

General
n:  11
Num of questions:  11

General
n:  6
Num of questions:  6

General
n:  2
Num of questions:  2

General
n:  3
Num of questions:  3

General
n:  2
Num of questions:  2

General
n:  3
Num of questions:  3

General
n:  2
Num of questions:  2

General
n:  2
Num of questions:  2

Genera

In [28]:
len(all_questions)

348

In [33]:
with open('all_questions.txt', 'w') as f:
    for question in all_questions:
        f.write(question)
        f.write('\n')

**Generate answers**

In [29]:
titles_with_text['Foreword']

['ISO (the International Organization for Standardization) is a worldwide federation of national standards  bodies (ISO member bodies). The work of preparing International Standards is normally carried out  through ISO technical committees. Each member body interested in a subject for which a technical  committee has been established has the right to be represented on that committee. International  organizations, governmental and non-governmental, in liaison with ISO, also take part in the work.  ISO collaborates closely with the International Electrotechnical Commission (IEC) on all matters of  electrotechnical standardization. The procedures used to develop this document and those intended for its further maintenance are  described in the ISO/IEC Directives, Part 1. In particular the different approval criteria needed for the  different types of ISO documents should be noted. This document was drafted in accordance with the  editorial rules of the ISO/IEC Directives, Part 2 (see  www

In [30]:
sum = 0
for title in list(titles_with_text.keys()):
    for i in range(len(titles_with_text[title])):
        print(title)
        text_token_length =  num_tokens_from_string(titles_with_text[title][i], "cl100k_base")
        
        if text_token_length == 0:
            print()
            continue
            
        n = (text_token_length // 100)**2 + 2
        print('sum: ', sum, '\n')
        sum += n
print(sum)

Foreword
sum:  0 

Introduction

Scope
sum:  18 

Normative references
sum:  21 

Terms and definitions
sum:  23 

Context of the organization

Understanding the organization and its context
sum:  25 

Understanding the needs and expectations of interested parties
sum:  28 

Understanding the needs and expectations of interested parties
sum:  30 

Determining the scope of the quality management system
sum:  33 

Quality management system and its processes
sum:  39 

Leadership

Leadership and commitment

General
sum:  45 

General
sum:  63 

General
sum:  74 

General
sum:  80 

General
sum:  82 

General
sum:  85 

General
sum:  87 

General
sum:  90 

General
sum:  92 

General
sum:  94 

Customer focus
sum:  96 

Policy

Establishing the quality policy
sum:  98 

Communicating the quality policy
sum:  100 

Organizational roles, responsibilities and authorities
sum:  102 

Planning

Actions to address risks and opportunities
sum:  105 

Quality objectives and planning to achieve the

In [86]:
all_answers = []
counter = 204

for title in ['Control of production and service provision']:
    for i in range(len(titles_with_text[title])):
        print(title)
        
        n = get_number_of_questions_for_title(titles_with_text[title][i])
        if n == -1:
            print()
            continue
        print('n: ', n)

        questions_list = all_questions[counter : counter+n]
        
        answer_prompt = f"""
            Here is the list of questions:
            {questions_list}
            Answer all of this question separately using information from this text delimited with ``` only:
            text: ```{titles_with_text[title][i]}```
            If you can't find answer from given text return: No answer.
            Return only numerated question-answer pairs.
        """

        raw_answers = get_completion(answer_prompt)
        raw_answers += '\n\n'
        #raw_answers = re.sub(r'\n\d\.', '\n', raw_answers)
        print('-----------------------------------------------------', 
              raw_answers, 
              '------------------------------------------------------',
              '\n')
        
        pattern = '(\?\n(Answer: )?((.+\n?)+)\n\n)'
        splits = re.findall(pattern, raw_answers)
        
        prepared_answers = []
        print(splits)
        for split in splits:
            #print(split)
            prepared_answers.append(split[2])
        #print()

        print('Num of answers: ', len(prepared_answers))
       
        #if(n != len(prepared_answers)):
        #print(raw_answers)
        print()

        all_answers += prepared_answers
        
        counter = counter+n

Control of production and service provision
n:  6
----------------------------------------------------- 1. What is the purpose of implementing production and service provision under controlled conditions?
Answer: The purpose of implementing production and service provision under controlled conditions is to ensure that the processes and outputs meet the criteria for control and the desired results are achieved.

2. What are the key elements of controlled conditions mentioned in the text?
Answer: The key elements of controlled conditions mentioned in the text are:
a) Availability of documented information defining product characteristics and desired results.
b) Availability and use of suitable monitoring and measuring resources.
c) Implementation of monitoring and measurement activities to verify control criteria and acceptance criteria.
d) Use of suitable infrastructure and environment for process operation.
e) Appointment of competent persons.
f) Validation and periodic revalidation of

In [87]:
raw_answers

'1. What is the purpose of implementing production and service provision under controlled conditions?\nAnswer: The purpose of implementing production and service provision under controlled conditions is to ensure that the processes and outputs meet the criteria for control and the desired results are achieved.\n\n2. What are the key elements of controlled conditions mentioned in the text?\nAnswer: The key elements of controlled conditions mentioned in the text are:\na) Availability of documented information defining product characteristics and desired results.\nb) Availability and use of suitable monitoring and measuring resources.\nc) Implementation of monitoring and measurement activities to verify control criteria and acceptance criteria.\nd) Use of suitable infrastructure and environment for process operation.\ne) Appointment of competent persons.\nf) Validation and periodic revalidation of the ability to achieve planned results.\ng) Implementation of actions to prevent human error

In [88]:
pattern = '(\?\n(Answer: )?((.+\n?)+)\n\n)'
splits = re.findall(pattern, raw_answers)
print(len(splits))

6


In [89]:
all_answers = []
counter = 0

for title in list(titles_with_text.keys()):
    for i in range(len(titles_with_text[title])):
        print(title)
        
        n = get_number_of_questions_for_title(titles_with_text[title][i])
        if n == -1:
            print()
            continue
        print('n: ', n)

        questions_list = all_questions[counter : counter+n]
        
        answer_prompt = f"""
            Here is the list of questions:
            {questions_list}
            Answer all of this question separately using information from this text delimited with ``` only:
            text: ```{titles_with_text[title][i]}```
            If you can't find answer from given text return: No answer.
            Return only numerated question-answer pairs.
        """

        raw_answers = get_completion(answer_prompt)
        raw_answers += '\n\n'
        #print(raw_answers, '\n')
        
        pattern = '(\?\n(Answer: )?((.+\n?)+)\n\n)'
        splits = re.findall(pattern, raw_answers)
        
        prepared_answers = []
        for split in splits:
            #print(split[2])
            prepared_answers.append(split[2])
        #print()

        print('Num of answers: ', len(prepared_answers))
       
        if(n != len(prepared_answers)):
            print(raw_answers)
        print()

        all_answers += prepared_answers
        
        counter = counter+n

Foreword
n:  18
Num of answers:  18

Introduction

Scope
n:  3
Num of answers:  3

Normative references
n:  2
Num of answers:  2

Terms and definitions
n:  2
Num of answers:  2

Context of the organization

Understanding the organization and its context
n:  3
Num of answers:  3

Understanding the needs and expectations of interested parties
n:  2
Num of answers:  2

Understanding the needs and expectations of interested parties
n:  3
Num of answers:  3

Determining the scope of the quality management system
n:  6
Num of answers:  6

Quality management system and its processes
n:  6
Num of answers:  6

Leadership

Leadership and commitment

General
n:  18
Num of answers:  18

General
n:  11
Num of answers:  11

General
n:  6
Num of answers:  6

General
n:  2
Num of answers:  2

General
n:  3
Num of answers:  3

General
n:  2
Num of answers:  2

General
n:  3
Num of answers:  3

General
n:  2
Num of answers:  2

General
n:  2
Num of answers:  2

General
n:  2
Num of answers:  2

Customer

In [92]:
all_answers

['ISO (the International Organization for Standardization) is a worldwide federation of national standards bodies (ISO member bodies).',
 'The purpose of ISO is to develop International Standards.',
 'International Standards are prepared through ISO technical committees.',
 'Each member body interested in a subject for which a technical committee has been established has the right to be represented on that committee.',
 'International organizations, governmental and non-governmental, collaborate with ISO.',
 'ISO collaborates closely with the International Electrotechnical Commission (IEC) on all matters of electrotechnical standardization.',
 'The procedures for developing ISO documents can be found in the ISO/IEC Directives, Part 1.',
 'The different approval criteria needed for the different types of ISO documents should be noted.',
 'There is a possibility that some elements of this document may be the subject of patent rights, and ISO shall not be held responsible for identifying 

In [95]:
with open('all_answers.txt', 'w', encoding='UTF-8') as f:
    for answer in all_answers:
        f.write(answer)
        f.write('\n\n')

In [96]:
qa_df = pd.DataFrame(columns = ['question', 'answer'], data = zip(all_questions, all_answers))
qa_df

,question,answer
0,What is ISO?,ISO (the International Organization for Standa...
1,What is the purpose of ISO?,The purpose of ISO is to develop International...
2,How are International Standards prepared?,International Standards are prepared through I...
3,Who can be represented on ISO technical commit...,Each member body interested in a subject for w...
4,Which organizations collaborate with ISO?,"International organizations, governmental and ..."
...,...,...
343,"How does the inclusion of ""services"" highlight...","The inclusion of ""services"" in ISO 9001:2008 h..."
344,Can conformity to requirements be confirmed be...,No answer.
345,What factors should an organization consider w...,The organization should consider factors such ...
346,Where are the requirements for applicability a...,The requirements for applicability are address...


In [97]:
qa_df.to_csv('qa_train_data.csv', index=False)

In [98]:
titles_with_text['Foreword']

['ISO (the International Organization for Standardization) is a worldwide federation of national standards  bodies (ISO member bodies). The work of preparing International Standards is normally carried out  through ISO technical committees. Each member body interested in a subject for which a technical  committee has been established has the right to be represented on that committee. International  organizations, governmental and non-governmental, in liaison with ISO, also take part in the work.  ISO collaborates closely with the International Electrotechnical Commission (IEC) on all matters of  electrotechnical standardization. The procedures used to develop this document and those intended for its further maintenance are  described in the ISO/IEC Directives, Part 1. In particular the different approval criteria needed for the  different types of ISO documents should be noted. This document was drafted in accordance with the  editorial rules of the ISO/IEC Directives, Part 2 (see  www

In [99]:
qa_df.iloc[17]['question']

'What is the URL for the ISO/IEC Directives?'

In [100]:
qa_df.iloc[17]['answer']

'The URL for the ISO/IEC Directives is www.iso.org/directives.'

In [101]:
len(titles_with_text['Foreword'][0])-41

2302

In [102]:
sentence_list = titles_with_text['Foreword'][0].split('. ')[:-1]
sum = 0
for sentence in sentence_list:
    print(len(sentence))
    sum += len(sentence)
avg = sum / len(sentence_list)
print('Average sentence len: ', avg)

132
103
144
112
134
140
110
130
117
79
191
126
297
160
247
80
Average sentence len:  143.875


In [103]:
len(titles_with_text['Terms and definitions'][0])

91

In [104]:
91 / 143.875

0.63249348392702

In [105]:
titles_with_text['Terms and definitions'][0]

'For the purposes of this document, the terms and definitions given in ISO 9000:2015 apply. '

**Finetune model**

In [16]:
qa_df = pd.read_csv('./qa_train_data.csv')
qa_df

,question,answer
0,What is ISO?,ISO (the International Organization for Standa...
1,What is the purpose of ISO?,The purpose of ISO is to develop International...
2,How are International Standards prepared?,International Standards are prepared through I...
3,Who can be represented on ISO technical commit...,Each member body interested in a subject for w...
4,Which organizations collaborate with ISO?,"International organizations, governmental and ..."
...,...,...
343,"How does the inclusion of ""services"" highlight...","The inclusion of ""services"" in ISO 9001:2008 h..."
344,Can conformity to requirements be confirmed be...,No answer.
345,What factors should an organization consider w...,The organization should consider factors such ...
346,Where are the requirements for applicability a...,The requirements for applicability are address...


In [37]:
output_dir = 'EleutherAI-pythia-410m-trained'

In [38]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='qa_train_data.csv',
    block_size=128
)
train_dataset

C:\Users\mihai\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [39]:
data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer,
  mlm=False
)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-410m', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, norma

In [43]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=3,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="no",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [44]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
base_answer = """The answer is simple: it’s not about the leader. It’s about the leader’s leadership.

Leadership is about the leader’s commitment to the mission.

Leadership is about the leader’s commitment to the people.

Leadership is about the leader’s commitment to the organization.

Leadership is about the leader’s commitment to the organization’s mission.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the organization’s culture.

Leadership is about the leader’s commitment to the
"""

In [38]:
training_output = trainer.train()

Step,Training Loss,Validation Loss


In [39]:
training_output

TrainOutput(global_step=99, training_loss=0.5676323253699024, metrics={'train_runtime': 268.0905, 'train_samples_per_second': 1.488, 'train_steps_per_second': 0.369, 'total_flos': 107607396188160.0, 'train_loss': 0.5676323253699024, 'epoch': 2.98})

In [40]:
question = 'How should top leadership demonstrate leadership and commitment?'

In [41]:
answer_6_epochs = print(inference(question, base_model, tokenizer))
answer_6_epochs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


,"Top leadership should demonstrate leadership and commitment by demonstrating that they are aware of the need for change and are prepared to take it. They should also demonstrate commitment to the organization's purpose and strategic direction."
How should top management demonstrate leadership and commitment in the quality management system?,Top management should demonstrate leadership and commitment by demonstrating that they are aware of the need for change and are prepared to take it. They should also demonstrate commitment to the organization's purpose and strategic direction.
What is the importance of having a quality management system that is up-to-date and effective?,The importance of having a quality management system that is up-to-date and effective lies in ensuring that it provides a sound basis for achieving conformity of products and services and achieving customer satisfaction.
How should the organization ensure that its quality management system is up-to-date and effecti

In [42]:
training_output = trainer.train()

Step,Training Loss,Validation Loss


In [43]:
training_output

TrainOutput(global_step=99, training_loss=0.1682375224702286, metrics={'train_runtime': 323.5629, 'train_samples_per_second': 1.233, 'train_steps_per_second': 0.306, 'total_flos': 107607396188160.0, 'train_loss': 0.1682375224702286, 'epoch': 2.98})

In [44]:
question = 'How should top leadership demonstrate leadership and commitment?'

In [45]:
answer_9_epochs = print(inference(question, base_model, tokenizer))
answer_9_epochs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


,"Top leadership should demonstrate leadership and commitment by demonstrating that they are committed to the quality of their work and the interests of the organization. They should also demonstrate competence, including the ability to demonstrate competence in a challenging environment."
How should top leadership demonstrate competence?,Top leadership should demonstrate competence by demonstrating the ability to demonstrate competence in a challenging environment.
What is the importance of demonstrating competence in a challenging environment in the quality management system?,The importance of demonstrating competence in a challenging environment in the quality management system is important in order to ensure that the quality of products and services can be guaranteed.
How should top leadership demonstrate leadership and commitment in the quality management system?,Top leadership should demonstrate leadership and commitment by demonstrating that they are committed to the quality of 